In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
from IPython.display import display, Markdown

import arcpy

#import arcgis libraries
from arcgis.gis import *
gis = GIS()

split_length = 0.125

In [2]:
def delIfExists(obj):
    try:
        if arcpy.Exists(obj):
            arcpy.Delete_management(obj)
            #print ("DELTED: " + obj)
        #else:
            #print ("DOESN'T EXIST: " + obj)
    except:
        #print ("PASSED: " + obj)
        pass

In [3]:
working_directory = os.getcwd()
data_folder         = os.path.join(working_directory, "data"        )
intermediate_folder = os.path.join(working_directory, "intermediate")
results_folder      = os.path.join(working_directory, "results"     )

print(data_folder)
print(intermediate_folder)
print(results_folder)

processGdb = os.path.join(intermediate_folder,'process.gdb')
processGdb

e:\WebAppBuilderForArcGIS_v2.20\server\apps\15\_dataprep\data
e:\WebAppBuilderForArcGIS_v2.20\server\apps\15\_dataprep\intermediate
e:\WebAppBuilderForArcGIS_v2.20\server\apps\15\_dataprep\results


'e:\\WebAppBuilderForArcGIS_v2.20\\server\\apps\\15\\_dataprep\\intermediate\\process.gdb'

In [5]:
# GET PROJECTS FROM WEB FEATURE CLASSES

delIfExists(os.path.join(processGdb,'_rtp_proj_points'))
delIfExists(os.path.join(processGdb,'_rtp_proj_lines' ))

point_url_feature_class = "https://services1.arcgis.com/taguadKoI1XFwivx/arcgis/rest/services/Final_2023_RTP_Projects/FeatureServer/0"
line_url_feature_class = "https://services1.arcgis.com/taguadKoI1XFwivx/arcgis/rest/services/Final_2023_RTP_Projects/FeatureServer/1"

arcpy.FeatureClassToFeatureClass_conversion(point_url_feature_class, processGdb, '_rtp_proj_points')
arcpy.FeatureClassToFeatureClass_conversion(line_url_feature_class , processGdb, '_rtp_proj_lines' )

<Result 'e:\\WebAppBuilderForArcGIS_v2.20\\server\\apps\\15\\_dataprep\\intermediate\\process.gdb\\_rtp_proj_lines'>

In [7]:
from IPython.display import clear_output, display

# A TOGGLE TO SET SO THAT RUNNING NOTEBOOK AGAIN THIS PROCESS ISN'T REPEATED IF NOT NEEDED, ie if this is set to 'False' don't run
bSplitSegs = True

if bSplitSegs:

    _fcIn  = os.path.join(processGdb,'_rtp_proj_lines')
    _fcSeg = _fcIn + '_segs'

    _fcRtpLinesSplitPointsTmp = _fcIn + '_split_points_tmp'
    _fcRtpLinesSegsTmp = _fcIn + '_segs_tmp'

    delIfExists(_fcSeg)

    bFirst = True

    # Create a search cursor for the feature class
    fields = ["OID@", "SHAPE@", "gis_id", "Mode"]
    cursor = arcpy.da.SearchCursor(_fcIn, fields)

    # Get the total number of rows in the feature class
    count = arcpy.GetCount_management(_fcIn)[0]

    # need to loop through each polyline separately since there are overlapping projects and have to be treated separately
    # Loop through each polyline in the input feature class
    for i, row in enumerate(cursor):

        # Get the OID and shape of the current polyline
        oid      = row[0]
        polyline = row[1]
        gid      = row[2]
        mode     = row[3]
        
        # delete temp layers
        delIfExists(_fcRtpLinesSplitPointsTmp)
        delIfExists(_fcRtpLinesSegsTmp)


        # Generate points along the polyline at the specified interval
        arcpy.GeneratePointsAlongLines_management(polyline, _fcRtpLinesSplitPointsTmp, 'DISTANCE', Distance = str(split_length) + ' miles')
        
        # Split the polyline at the generated points
        arcpy.SplitLineAtPoint_management(polyline, _fcRtpLinesSplitPointsTmp, _fcRtpLinesSegsTmp, '5 feet')

        
        # Add a new field to the output feature class to store the original OID of the polyline
        arcpy.AddField_management(_fcRtpLinesSegsTmp, "gis_id", "TEXT")
        arcpy.AddField_management(_fcRtpLinesSegsTmp, "Mode"  , "TEXT")
        
        # Populate the ORIG_OID field with the OID of the original polyline
        with arcpy.da.UpdateCursor(_fcRtpLinesSegsTmp, ["gis_id","Mode"]) as update_cursor:
            for update_row in update_cursor:
                update_row[0] = gid
                update_row[1] = mode
                update_cursor.updateRow(update_row)

        if bFirst:
            arcpy.management.Copy(_fcRtpLinesSegsTmp, _fcSeg)
            bFirst = False
        else:
            arcpy.management.Append(_fcRtpLinesSegsTmp, _fcSeg)

        # Update the progressor
        progress = "Processing polylines. Done processing: " + str(i+1) + " of " + str(count)
        display(progress, display_id="progress")

        # Clear the previous progress message
        clear_output(wait=True)

    # Clean up the cursor and progressor
    del cursor


'Processing polylines. Done processing: 1109 of 1109'

In [8]:

# delete temp layers
delIfExists(_fcRtpLinesSplitPointsTmp)
delIfExists(_fcRtpLinesSegsTmp)